In [29]:
# pip install datasets 
# pip install pinecone-client
# pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
# pip install sentence-transformers

from datasets import load_dataset
import pandas as pd

MS_IS_all_modules = pd.read_excel("C:/Users/FelixNeubauer/ML_Kurs/enterprise-ai-project/extracted_final_modules/MS_IS_all_modules.xlsx")
pd_table_MS_IS_all_modules = MS_IS_all_modules.astype(str)


In [30]:
pd_table_MS_IS_all_modules.head()

,Module title,Abbreviation,Module coordinator,Module offered by,ETCS,Method of grading,Duration,Module level,Contents,Intended learning outcomes,Courses,Method of assessment,Allocation of places,Additional information,Workload,Teaching cycle,Referred to in LPO I
0,Information Processing within Organizations,12-IV-161-m01,Faculty of Business Management and Economics,holder of the Chair of Business Management and...,5,numerical grade,1 semester,graduate,\n\nContent:\nThis course provides students wi...,"After completing the course ""Integrated Inform...",\n\nV (2) + Ü (2)\n\n,\n\nwritten examination (approx. 60 minutes)\n...,\n\n--\n\n,\n\n--\n\n,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n
1,IT-Management,12-M-ITM-161-m01,Faculty of Business Management and Economics,Holder of the Chair of Information Systems Eng...,5,numerical grade,1 semester,graduate,\n\nContent:\nThis course provides students wi...,"After completing the course ""IT Management"", s...",\n\nV (2) + Ü (2)\n\n,\n\na) written examination (approx. 60 minutes...,\n\n--\n\n,\n\n--\n\n,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n
2,Project Seminar,12-PS-192-m01,Faculty of Business Management and Economics,Holder of the Chair of Business Management and...,15,numerical grade,1 semester,graduate,\n\nContent:\nIn small project teams of 4 to 1...,"After completing the course ""Projektseminar"", ...",\n\nS (2)\n\n,\n\nproject: preparing a conceptual design (ap...,\n\n--\n\n,\n\n--\n\n,\n\n450 h\n\n,\n\n--\n\n,\n\n--\n\n
3,Information Retrieval,10-I=IR-161-m01,Institute of Computer Science,Dean of Studies Informatik (Computer Science),5,numerical grade,1 semester,graduate,\n\nIR models (e. g. Boolean and vector space ...,The students possess theoretical and practical...,\n\nV (2) + Ü (2)\n\n,\n\nwritten examination (approx. 60 to 120 min...,\n\n--\n\n,\n\nFocuses available for students of the Mast...,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n
4,Analysis and Design of Programs,10-I=PA-161-m01,Institute of Computer Science,holder of the Chair of Computer Science II,5,numerical grade,1 semester,graduate,"\n\nProgram analysis, model creation in softwa...","The students are able to analyse programs, to ...",\n\nV (2) + Ü (2)\n\n,\n\nwritten examination (approx. 60 to 120 min...,\n\n--\n\n,\n\nFocuses available for students of the Mast...,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n


In [31]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the table embedding model from huggingface models hub
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever




SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [32]:
def _preprocess_tables (pd_table_MS_IS_all_modules):
    processed = []
    MS_IS_all_modules_csv = "\n".join([pd_table_MS_IS_all_modules.to_csv(index=False)])
    processed.append(MS_IS_all_modules_csv)
    return processed




### The formatted table may not make sense to us, but the embedding model is trained to understand it and generate accurate embeddings.

In [33]:
processed_modules = _preprocess_tables(pd_table_MS_IS_all_modules)

processed_modules[0]

'Module title,Abbreviation,Module coordinator,Module offered by,ETCS,Method of grading,Duration,Module level,Contents,Intended learning outcomes,Courses,Method of assessment,Allocation of places,Additional information,Workload,Teaching cycle,Referred to in LPO I\r\nInformation Processing within Organizations,12-IV-161-m01,Faculty of Business Management and Economics,holder of the Chair of Business Management and Business ,5,numerical grade,1 semester,graduate,"\n\nContent:\nThis course provides students with an in-depth overview of the structure and the application areas of business \nmanagement information systems in enterprises and public institutions.\n\nOutline of syllabus:\n1. What is software: concepts, categories, application\n2. Software life cycle: duration, phases, steps\n3. As-is analysis: tasks, problems\n4. To-be concept: system design, data design, dialog design, function design\n5. Object orientation: paradigm shift\n6. Change management: meaning, methodologies, project 

In [34]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key="1d3ebed2-b4fa-4523-9cd4-722ac005fc6b",
    environment="us-west1-gcp-free"
)

In [35]:
# you can choose any name for the index
index_name = "table-qa-module-catalogue"

# check if the table-qa index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to table-qa index we created
index = pinecone.Index(index_name)

In [36]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(processed_modules), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(processed_modules))
    # extract batch
    batch = processed_modules[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()
    # create unique IDs ranging from zero to the total number of tables in the dataset
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1}

In [37]:
query = "How many ETCS points is the module IT-Management worth?"
# generate embedding for the query
xq = retriever.encode([query]).tolist()
# query pinecone index to find the table containing answer to the query
result = index.query(xq, top_k=1)
result


{'matches': [{'id': '0', 'score': 0.561872363, 'values': []}], 'namespace': ''}

In [38]:
id = int(result["matches"][0]["id"])
# Since we only have one table/vector in the index, we know that the id of the table is 0 .. 

#### The table returned by the Pinecone index indeed has the answer to our query. Now we need a model that can read this table and extract the precise answer.

In [39]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

model_name = "google/tapas-base-finetuned-wtq"
# load the tokenizer and the model from huggingface model hub
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(model_name, local_files_only=False)
# load the model and tokenizer into a question-answering pipeline
pipe = pipeline("table-question-answering",  model=model, tokenizer=tokenizer, device=device)

In [45]:
# Let's run the table returned by the Pinecone index and the query we used before into the question-answering pipeline to extract the answer.

query1 = "How many ETCS are there for the project seminar?"

pipe(table=pd_table_MS_IS_all_modules, query=query1)

{'answer': 'AVERAGE > 15',
 'coordinates': [(2, 4)],
 'cells': ['15'],
 'aggregator': 'AVERAGE'}

In [41]:
def get_found_cells_from_table(table, query):
    # run the table and query through the question-answering pipeline
    cells = pipe(table=table, query=query).get("cells")
    return cells

In [42]:
#print (get_found_cells_from_table(pd_table_MS_IS_all_modules, query1))